In [ ]:
import torch
import numpy as np
import random
import pandas as pd

In [ ]:
# To make random weights in NN consistent from launch to launch on this device
# To actually distinguish an improvement
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [ ]:
# Importing wine dataset
import sklearn.datasets
wine = sklearn.datasets.load_wine()
wine.data.shape


(178, 13)

In [ ]:
print(wine.DESCR)
df = pd.DataFrame(wine.data, columns=wine.feature_names)
df.head()

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0


In [ ]:
from sklearn.model_selection import train_test_split

features = 13
X_train, X_test, y_train, y_test = train_test_split(
    wine.data[:,:features], # we consider all features
    wine.target,
    test_size = 0.3,
    shuffle=True
)
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
import torch.nn as nn
class WineNet(nn.Module):
  def __init__(self, n_input_neurons, n_hidden_neurons):
    super(WineNet, self).__init__()
    self.fc1 = nn.Linear(n_input_neurons, n_hidden_neurons)
    self.al1 = nn.Sigmoid()
    self.fc2 = nn.Linear(n_hidden_neurons, n_hidden_neurons)
    self.al2 = nn.Sigmoid()
    self.fc3 = nn.Linear(n_hidden_neurons, 3)
    self.sm = nn.Softmax(dim=1)
  
  def forward(self, x):
    x = self.fc1(x)
    x = self.al1(x)
    x = self.fc2(x)
    x = self.al2(x)
    x = self.fc3(x)
    return x
  def inference(self, x):
    x = self.forward(x)
    x = self.sm(x)
    return x

wineNet = WineNet(features, 5)

In [ ]:
loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(wineNet.parameters(), lr=1.0e-3)

In [ ]:
batch_size = 10
for epoch in range(10000):
  permutation = np.random.permutation(len(X_train))
  for i in range(0, len(X_train), batch_size):
    optimizer.zero_grad()
    x_batch = X_train[permutation[i:i+batch_size]]
    y_batch = y_train[permutation[i:i+batch_size]]
    prediction = wineNet.forward(x_batch)
    loss_value = loss(prediction, y_batch)
    loss_value.backward()
    optimizer.step()
  if epoch%100==0:
    test_preds = wineNet.forward(X_test)
    test_preds = test_preds.argmax(dim=1)
    print((test_preds == y_test).float().mean())

tensor(0.2407)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4074)
tensor(0.4

In [ ]:
%matplotlib inline 

plt.rcParams['figure.figsize'] = (10, 8)
n_classes = 3
plot_colors = ['g', 'orange', 'black']
plot_step = 0.02

x_min, x_max = X_train[:, 0].min() - 1, X_train[:, 0].max() + 1
y_min, y_max = X_train[:, 1].min() - 1, X_train[:, 1].max() + 1

xx, yy = torch.meshgrid(torch.arange(x_min, x_max, plot_step),
                        torch.arange(y_min, y_max, plot_step))

preds = wineNet.inference(
    torch.cat([xx.reshape(-1,1), yy.reshape(-1,1)], dim=1)
)


preds_class = preds.data.numpy().argmax(axis=1)
preds_class = preds_class.reshape(xx.shape)
plt.contourf(xx, yy, preds_class, cmap='Accent')

for i, color in zip(range(n_classes), plot_colors):
    indexes = np.where(y_train == i)
    plt.scatter(X_train[indexes, 0], 
                X_train[indexes, 1], 
                c=color, 
                label=wine.target_names[i],
                cmap='Accent')
    plt.xlabel(wine.feature_names[0])
    plt.ylabel(wine.feature_names[1])
    plt.legend()

NameError: ignored